In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances

from atlas_yao import *

In [2]:
data_dir = "data/klein"

Vecs = np.load(data_dir+"/klein_uniform_1000.npy")
#Vecs = np.load(data_dir+"/klein_uniform_100.npy")
patches_pos = np.load(data_dir+"/patches_pos_razor_big.npy")
patches_neg = np.load(data_dir+"/patches_neg_razor_big.npy")

vecs_pos = []
vecs_neg = []
for j in range(patches_pos.shape[0]):
    vecs_pos.append(patches_pos[j, :, :].reshape(9))
    vecs_neg.append(patches_neg[j, :, :].reshape(9))
Vecs_pos = np.vstack(vecs_pos)
Vecs_neg = np.vstack(vecs_neg)

print(Vecs_pos.shape)
print(Vecs_neg.shape)

(1000, 9)
(1000, 9)


In [3]:
# Number of charts inspired by previous notebook
n_charts = 64
km_max_iter = 1000
grid_len = 30

ka = atlas_yao(Vecs, Vecs_pos, Vecs_neg, n_charts,
              km_max_iter=km_max_iter,
              grid_len=grid_len,
              load_dist_mat=True)

Getting graph as sparse matrix...
Done
Getting graph from sparse matrix...
Done
Done


  5%|▍         | 3/64 [00:00<00:02, 22.73it/s]

Constructing enormous graph for brute-force geodesic approximation.
Trying eps = 0.6


100%|██████████| 41088/41088 [05:39<00:00, 121.13it/s]


In [4]:
# Save atlas
ka.store_atlas("stored_charts/oct_20_2023/chart")

In [5]:
# Get initial points for RPB
### Get random positive point
np.random.seed(600)
n_pos = Vecs_pos.shape[0]
ind_pos = np.random.randint(n_pos)
x_pos = Vecs_pos[ind_pos, :]
### Get closest negative point
X_pos = x_pos.reshape((1, 9))
dist_vec = euclidean_distances(X_pos, Vecs_neg)[0, :]
n_neg = Vecs_neg.shape[0]
ind_neg = np.random.randint(n_neg)
x_neg = Vecs_neg[ind_neg, :]

# Ingest x_pos, x_neg into atlas graph coordinates
xi_pos, chart_pos = ka.ingest_ambient_point(x_pos)
xi_neg, chart_neg = ka.ingest_ambient_point(x_neg)

In [6]:
n_iters = 10
dist_max = 2.0
"""
output = ka.riemannian_principal_boundary_mod(chart_init_pos=(4, 0),
                                        chart_init_neg=(4, 4),
                                        n_iters=n_iters,
                                        dist_max=dist_max)
"""
output =  ka.riemannian_principal_boundary(xi_pos,
                                           chart_pos,
                                           xi_neg,
                                           chart_neg,
                                           stepsize=0.1,
                                           n_iters=n_iters,
                                           dist_max=dist_max)

100%|██████████| 10/10 [54:53<00:00, 329.35s/it]


In [7]:
# Save RPB results
xi_pos_list, chart_pos_list, xi_prime_pos_list, xi_neg_list, chart_neg_list, xi_prime_neg_list, xi_bou_list, chart_bou_list, xi_prime_bou_list, xi_prime_bou_pos_list, xi_prime_bou_neg_list = output

names = ["xi_pos_list", "chart_pos_list", "xi_prime_pos_list", "xi_neg_list", "chart_neg_list", "xi_prime_neg_list", "xi_bou_list", "chart_bou_list", "xi_prime_bou_list", "xi_prime_bou_pos_list", "xi_prime_bou_neg_list"]

temp_dir = "temp_save_"+str(n_iters)
for item, name in zip(output, names):
    np.save(temp_dir+"/"+name+".npy", item)